# TM10007 Assignment template

In [4]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [5]:
# Data loading functions. Uncomment the one you want to use
#from worcgist.load_data import load_data
from worclipo.load_data import load_data
#from worcliver.load_data import load_data
#from ecg.load_data import load_data

# Import classifiers
from sklearn import model_selection

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

X = data.drop("label",axis=1)
y = data["label"]

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y)


The number of samples: 115
The number of columns: 494


### Preprocessing

Missing data
* Part 1: Finding missing data

In [8]:
# Define missing value indicators
custom_missing = ['NA', 'null', 'N/A', '?', 'None', 'none', '-']

# Count NaNs
nan_counts = X_train.isna().sum()
nan_counts_selected = nan_counts[nan_counts !=0]

# Count empty strings
empty_string_counts = (X_train == '').sum()
empty_string_counts_selected = empty_string_counts[empty_string_counts != 0]

# Count zero's
zero_counts = (X_train == 0).sum()
zero_counts_selected = zero_counts[zero_counts != 0]

# Count custom missing indicators (case-insensitive match)
custom_missing_counts = X_train.apply(lambda col: col.astype(str).str.lower().isin([val.lower() for val in custom_missing]).sum())
custom_missing_counts_selected = custom_missing_counts[custom_missing_counts !=0]

# Printing
print(nan_counts_selected)
print(empty_string_counts_selected)
print(zero_counts_selected)
print(custom_missing_counts_selected)


Series([], dtype: int64)
Series([], dtype: int64)
PREDICT_original_sf_area_min_2.5D                                       2
PREDICT_original_tf_LBP_quartile_range_R8_P24                           4
PREDICT_original_tf_LBP_peak_R15_P36                                    1
PREDICT_original_tf_LBP_peak_position_R15_P36                           1
PREDICT_original_vf_Frangi_full_quartile_range_SR(1.0, 10.0)_SS2.0      2
PREDICT_original_vf_Frangi_edge_quartile_range_SR(1.0, 10.0)_SS2.0      2
PREDICT_original_vf_Frangi_inner_quartile_range_SR(1.0, 10.0)_SS2.0     5
PREDICT_original_phasef_monogenic_peak_WL3_N5                           4
PREDICT_original_phasef_monogenic_peak_position_WL3_N5                  4
PREDICT_original_phasef_phasecong_quartile_range_WL3_N5                10
PREDICT_original_phasef_phasesym_quartile_range_WL3_N5                  6
dtype: int64
Series([], dtype: int64)


* Part 2: Processing missing data

In [9]:
# Missing data was only find in the form of zero's, therefore only zero_counts is further used.
limit = len(data.index)*20/100 #If 20% or more of the data wihtin one feature is missing the feature is deleted
valid_columns = [col for col, count in zero_counts.items() if count < limit]

# Keep only the valid columns in both X_train and X_test
X_train = X_train[valid_columns]
X_test = X_test[valid_columns]

# Imputate remaining zero's
# Import
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from scipy.stats import shapiro

# Replace zeros with NaN
X_train.replace(0, np.nan, inplace=True)
X_test.replace(0, np.nan, inplace=True)

imputation_strategies = {}  # Store mean/median decision per column

for col in X_train.select_dtypes(include=['number']).columns:  # Only numeric columns
    col_data = X_train[col].dropna()  # Remove NaN values for testing

    if len(col_data) > 3:  # Shapiro requires at least 3 non-null values
        if col_data.nunique() == 1:  # Check if all values are the same
            strategy = 'median'  # Default to median if no variability
        else:
            _, p = shapiro(col_data)
            strategy = 'mean' if p > 0.05 else 'median'
    else:
        strategy = 'median'  # Default to median if too few values

    imputation_strategies[col] = strategy

# Create imputers for mean and median
mean_imputer = SimpleImputer(strategy='mean')
median_imputer = SimpleImputer(strategy='median')

for col, strategy in imputation_strategies.items():
    imputer = mean_imputer if strategy == 'mean' else median_imputer
    X_train[[col]] = imputer.fit_transform(X_train[[col]])
    X_test[[col]] = imputer.transform(X_test[[col]])  # Use the same imputer

Feature extraction

Lloyd

Inge

Scaling data